In [1]:
import os
import mido
import utils ##
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

Datos preprocesados:
https://drive.google.com/drive/folders/1jzfK-VM_Qb76K31Y7iMpsi1JeQ7QqU5W?usp=sharing

# Importar Datos

In [2]:
df_midis_info = pd.read_csv('datos_procesados/midis/midis_info.csv', index_col=0)
df_midis_notas = pd.read_csv('datos_procesados/midis/midis_notas.csv', index_col=0)

In [3]:
# Tipos de MIDI
df_non_standard = pd.read_csv('datos_procesados/midis/tipos/no_estandar.csv', index_col=0)
df_standard = pd.read_csv('datos_procesados/midis/tipos/estandar.csv', index_col=0)

In [4]:
# Usar los audios con expresión MIDI estándar
df_midi = df_standard.merge(df_midis_info, how='left', left_index=True, right_index=True)
df_midi = df_midi.drop(columns=['problem?'])
df_midi = df_midi.rename({'velocity':'velocity_start'}, axis=1)

df_notas = df_midis_notas.loc[df_midi.index]

# MIDIS

In [5]:
lowest_time = df_midi.min_time.min()
print(lowest_time, 's')

0.125 s


### Filtrar frecuencias
Sólo se conservan aquellas que son alcanzables por una voz humana

In [6]:
### Filtra notas que no son ejecutadas
notas_mask = (df_notas.sum() > 0)
notas_ejecutadas = notas_mask[notas_mask]

# Transforma en DataFrame
notas_ejecutadas = notas_ejecutadas.index.str.extract('(^\d+) \((\w#?\d)\)') # Separa la clave MIDI del Cifrado
notas_ejecutadas = notas_ejecutadas.rename({0:'midi-ET', 1: 'cifrado'}, axis=1) # Renombrar
notas_ejecutadas['midi-ET'] = notas_ejecutadas['midi-ET'].astype(int) # Regresar a numérico
notas_ejecutadas['herts-ET'] = utils.preprocess.nota_a_frequencia(notas_ejecutadas['midi-ET']) # Convertir a frecuencia

In [7]:
# Calcular cotas de filtrado
hz_std = notas_ejecutadas['herts-ET'].std() 

hz_lb = np.floor( notas_ejecutadas['herts-ET'].min() * (1-0.5) )
hz_ub = np.ceil( notas_ejecutadas['herts-ET'].max() + 0.5*hz_std )

freq_range = (hz_lb, hz_ub)
print(f'Rango de frecuencias utilizable (Hz): {freq_range}')

Rango de frecuencias utilizable (Hz): (18.0, 1167.0)


### Reducción dimensión de tiempo

La reducción se realiza a un tiempo $T$ siguiendo los siguientes pasos para garantizar la integridad de las notas:

- Recortar tiempo muerto al inicio del MIDI
- Encontrar el punto de corte en $T$ del MIDI
- Verificar si hay una nota sonando
- Eliminar la nota sonando
- Capturar el tiempo $t_f$ en la que termina la última nota ($t_f \leq 10$)

In [23]:
def procesar_midis(midi_names, output_folder, T):
    """
    Aplica las funciones de recorte de desfase de tiempo inicial (lstrip) y ajuste a T segundos 
    a todos los archivos MIDI en una carpeta.
    
    Parámetros:
        midi_names (Series): Nombres de los MIDI a convertir.
        output_folder (str): Carpeta de destino para guardar los archivos MIDI procesados.
        T (float): Duración máxima en segundos para los archivos MIDI procesados.
    """
    # Revisar carpeta de salida
    os.makedirs(output_folder, exist_ok=True)
    # Definir carpeta de MIDIs
    input_folder = 'datos/MIDIs/midi_data/'

    
    for midi_name in midi_names:
        midi_path = os.path.join(input_folder, midi_name + '.mid')
        mid = mido.MidiFile(midi_path)
        
        # Recortar tiempo muerto
        mid = utils.midi.lstrip(mid)
        
        # Recortar duración a T segundos
        tempo = df_midi['tempo_ms'].loc[midi_name]
        mid = utils.midi.trim(mid, tempo, T)
        
        # Guardar el archivo MIDI procesado
        output_path = os.path.join(output_folder, midi_name + '.mid')
        mid.save(output_path)


In [24]:
procesar_midis(midi_names=df_midi.index,
               output_folder='datos_procesados/midis/trimmed/',
               T=10)

### MIDI 2 vector

In [8]:
def crear_vectores(midi_names, output_folder, N, T):
    """
    Convierte todos los midi ingresados a vector, siguiendo los criterios de asignación:
    `vec[i] -> comienzo del evento`
    `vec[i+1] -> Duración L_segs del evento` 
    Esto implica que si una nota deja de sonar en la casilla [i], esa casilla se marcará con "0", 
    pues se considera que consumió la casilla i-1 y terminó al inicio de la casilla i.
    
    Parámetros:
        midi_names (Series): Nombres de los MIDI a convertir.
        output_folder (str): Carpeta de destino para guardar los vectores MIDI.
        N (int): Longitud del vector MIDI (normalmente del tamaño de ventanas temporales)
        T (float): Duración máxima en segundos que codifica el vector
    """
    # Revisar carpeta de salida
    os.makedirs(output_folder, exist_ok=True)
    # Definir carpeta de MIDIs
    input_folder = 'datos_procesados/midis/trimmed/'

    
    for midi_name in midi_names:
        midi_path = os.path.join(input_folder, midi_name + '.mid')
        mid = mido.MidiFile(midi_path)
        
        # Transformar a vector
        vector = utils.midi.midi2vec(mid, N, T)
        
        # Guardar vector
        output_vect_path = os.path.join(output_folder, midi_name + '_target.npy')
        np.save(output_vect_path, vector)


In [ ]:
crear_vectores(midi_names=df_midi.index,
               output_folder='datos_procesados/midis/target_vectors/',
               N=int(10 / (lowest_time/2)),
               T=10)

- Dimensión de vectores

El máximo debe ser de 160 pues $\text{sr} / (\text{lowest\_time} / 2) = 160$. Las que son menores a este valor, es debido a que los audios son más cortos.

In [10]:
folder_path = 'datos_procesados/midis/target_vectors/'
names = []
shapes = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    data = np.load(file_path)
    names.append(filename)
    shapes.append(data.shape)

In [11]:
window_shapes = pd.DataFrame(shapes, index = names)
window_shapes.describe()

,0
count,10053.000000
mean,143.427932
std,12.131464
min,80.000000
25%,137.000000
50%,145.000000
75%,152.000000
max,160.000000


# Tarareos


In [17]:
def procesar_tarareos(audio_names, output_folder_frames, output_folder_espectros, L_seg, freq_range):
    """
    Procesa todos los archivos de audio seleccionados, aplicando recorte, 
    división en ventanas y generación de espectrogramas.
    
    Parámetros:
        audio_names (Series): Nombres de los audios a procesar.
        output_folder_frames (str): Carpeta para guardar los arreglos de ventanas de audio procesados.
        output_folder_espectros (str): Carpeta para guardar los espectrogramas generados.
        L_seg (float): Duración de cada ventana en segundos.
        freq_range (tuple): Rango de frecuencias a considerar (min_freq, max_freq).
    """
    # Librería de transformación de tarareos
    audio = utils.tarareos
    # Crear carpetas de salida si no existen
    os.makedirs(output_folder_frames, exist_ok=True)
    os.makedirs(output_folder_espectros, exist_ok=True)
    # Definir carpeta de tarareos
    input_folder = 'datos/Tarareos/wav_data_sync_with_midi/'
    midi_folder = 'datos_procesados/midis/trimmed/' # Los ya procesados
    final_freqs = {}

    for audio_name in audio_names:
        audio_path = os.path.join(input_folder, audio_name + '.wav')
        y, sr = librosa.load(audio_path)
        mid = mido.MidiFile(midi_folder + audio_name + '.mid')
        T_midi = mid.length

        # Recortar el audio
        audio_recortado = audio.trim(y, sr, T_midi, top_db=55) #, top_db=-55
        
        ### VENTANAS
        audio_frames, L = audio.dividir_en_ventanas(audio_recortado, sr, L_seg)
        
        output_frame_path = os.path.join(output_folder_frames, audio_name + '_frames.npy')
        # print(audio_frames.shape)
        np.save(output_frame_path, audio_frames)
        
        ### ESPECTROGRAMA
        espectrograma, _freqs = audio.espectrograma(audio_recortado, sr, L, freq_range)
        final_freqs[audio_name] = _freqs
    
        output_espectro_path = os.path.join(output_folder_espectros, audio_name + '_espectrograma.npy')
        # print(espectrograma.shape)
        np.save(output_espectro_path, espectrograma)

In [ ]:
procesar_tarareos(audio_names=df_midi.index,
                  output_folder_frames='datos_procesados/tarareos/ventanas/',
                  output_folder_espectros='datos_procesados/tarareos/espectrogramas/',
                  L_seg=lowest_time/2,
                  freq_range=freq_range)

In [257]:
#### Muestras
# T_audio > 10 : F04_0300_0001_1_D
# T_audio = 10 : F01_0009_0001_2_D
# T_audio < 10 : F01_0035_0001_2_D
# Nota sonando en T : F01_0028_0003_2_D
# Nota no sonando en T: F01_0022_0001_1_D

- Dimensiones de ventanas

In [15]:
folder_path = 'datos_procesados/tarareos/ventanas/'
names = []
shapes = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    data = np.load(file_path)
    names.append(filename)
    shapes.append(data.shape)

In [19]:
window_shapes = pd.DataFrame(shapes, index = names)
window_shapes.describe()

,0,1
count,10053.0,10053.000000
mean,1378.0,150.395106
std,0.0,10.921703
min,1378.0,73.000000
25%,1378.0,146.000000
50%,1378.0,154.000000
75%,1378.0,157.000000
max,1378.0,230.000000


- Dimensiones de espectrogramas

In [20]:
folder_path = 'datos_procesados/tarareos/espectrogramas/'
names = []
shapes = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    data = np.load(file_path)
    names.append(filename)
    shapes.append(data.shape)

In [21]:
espectro_shapes = pd.DataFrame(shapes, index = names)
espectro_shapes.describe()

,0,1
count,10053.0,10053.000000
mean,70.0,151.395106
std,0.0,10.921703
min,70.0,74.000000
25%,70.0,147.000000
50%,70.0,155.000000
75%,70.0,158.000000
max,70.0,231.000000


# Otros

In [189]:
midi = mido.MidiFile('datos_procesados/midis/trimmed/F04_0300_0001_1_D.mid')
midi

MidiFile(type=1, ticks_per_beat=1024, tracks=[
  MidiTrack([
    MetaMessage('set_tempo', tempo=857143, time=0),
    MetaMessage('key_signature', key='C', time=0),
    MetaMessage('time_signature', numerator=2, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('end_of_track', time=0)]),
  MidiTrack([
    MetaMessage('track_name', name='é\x92¢ç\x90´, Piano', time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('pitchwheel', channel=0, pitch=0, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('note_on', channel=0, note=65, velocity=80, time=0),
    Message('note_off', channel=0, note=65, velocity=0, time=768),
    Message('note_on', channel=0, note=62, velocity=80, time=0),
    Message('note_off', channel=0, note=62, velocity=0, time=256),
    Message('note_on', channel=0, note=60, velocity=80, time=0),
    Message('note_off'

In [181]:
np.ceil(midi.length / L_segs)

158.0

In [190]:
tick_nota = 512
print(mido.tick2second(tick_nota,1024,857143))

L_segs = lowest_time/2
N = int( np.ceil(midi.length / L_segs) )
midi2vec(midi, N=N, T=10)

0.4285715
0.06329113924050633


[1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2]

In [191]:
len(midi2vec(midi, N=N, T=10))

0.06329113924050633


144

In [157]:
(6)*L_segs +  L_segs

0.4375

In [143]:
np.linspace(0, 10, 160+1)[:146]

array([0.    , 0.0625, 0.125 , 0.1875, 0.25  , 0.3125, 0.375 , 0.4375,
       0.5   , 0.5625, 0.625 , 0.6875, 0.75  , 0.8125, 0.875 , 0.9375,
       1.    , 1.0625, 1.125 , 1.1875, 1.25  , 1.3125, 1.375 , 1.4375,
       1.5   , 1.5625, 1.625 , 1.6875, 1.75  , 1.8125, 1.875 , 1.9375,
       2.    , 2.0625, 2.125 , 2.1875, 2.25  , 2.3125, 2.375 , 2.4375,
       2.5   , 2.5625, 2.625 , 2.6875, 2.75  , 2.8125, 2.875 , 2.9375,
       3.    , 3.0625, 3.125 , 3.1875, 3.25  , 3.3125, 3.375 , 3.4375,
       3.5   , 3.5625, 3.625 , 3.6875, 3.75  , 3.8125, 3.875 , 3.9375,
       4.    , 4.0625, 4.125 , 4.1875, 4.25  , 4.3125, 4.375 , 4.4375,
       4.5   , 4.5625, 4.625 , 4.6875, 4.75  , 4.8125, 4.875 , 4.9375,
       5.    , 5.0625, 5.125 , 5.1875, 5.25  , 5.3125, 5.375 , 5.4375,
       5.5   , 5.5625, 5.625 , 5.6875, 5.75  , 5.8125, 5.875 , 5.9375,
       6.    , 6.0625, 6.125 , 6.1875, 6.25  , 6.3125, 6.375 , 6.4375,
       6.5   , 6.5625, 6.625 , 6.6875, 6.75  , 6.8125, 6.875 , 6.9375,
      